In [1]:
!pip install redis-py-cluster

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 KB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.1/72.1 KB 12.2 MB/s eta 0:00:00
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


#### Without the VPC

In [78]:
from rediscluster import RedisCluster
import logging

#testredis.m7ovi1.ng.0001.use1.cache.amazonaws.com:6379

logging.basicConfig(level=logging.INFO)
redis_client = RedisCluster(
    # - both work
    #startup_nodes=[{"host": 'testblogredismemcache.m7ovi1.ng.0001.use1.cache.amazonaws.com', "port": '6379'}],
    startup_nodes=[{"host": 'testredis.m7ovi1.ng.0001.use1.cache.amazonaws.com', "port": '6379'}],
    decode_responses=True, init_slot_cache=False, #skip_full_coverage_check=True ,
    username='testRedisUser', 
    password='testRedisUserpassword'
)
print("cluster connected")
if redis_client.ping():
    print("Connected to Redis")

INFO:rediscluster.client:Created new instance of RedisCluster client instance
INFO:rediscluster.client:Using ClusterConnectionPool


cluster connected


#### With the Default account VPC

In [4]:
from redis import Redis
import logging
redis_client = Redis(
    # - vpce-0ee05c94e950bcd3c-hafv9vy6.elasticache.us-east-1.vpce.amazonaws.com
    # - elasticache.us-east-1.amazonaws.com
    # - 'testredis.m7ovi1.ng.0001.use1.cache.amazonaws.com'
    host='testredis.m7ovi1.ng.0001.use1.cache.amazonaws.com', 
    port=6379, #decode_responses=True, ssl=True, 
    username='default', #'testredisuser', 'default'
    #password='testRedisUserPassword'
)
# redis_client = Redis(
#     host='testredis.m7ovi1.ng.0001.use1.cache.amazonaws.com',
#     username='testredisuser',
#     password='testRedisUserPassword', ssl_cert_reqs=None,  decode_responses=True)

print("Redis client starting to connect ")
if redis_client.ping():
    logging.info("Connected to Redis")

Redis client starting to connect 


In [5]:
redis_client

Redis<ConnectionPool<Connection<host=testredis.m7ovi1.ng.0001.use1.cache.amazonaws.com,port=6379,db=0>>>

In [12]:
import json
import uuid

def get():
    key_mask = "customer:*"
    customers = []
    for key in redis_client.scan_iter(key_mask):
        customer_id = key #key.split(':')[1]
        customer = redis_client.hgetall(key)
        customer['id'] = customer_id
        customers.append(customer)
        #print(customer)
    return customers

def post(json_struct):
    print(json_struct)
    json_str = json.dumps(json_struct)
    
    customer_id = json_struct['customer_id'] #str(uuid.uuid4())
    key = "customer:" + customer_id
    print(f"key={key}")
    write_redis = redis_client.hset(key, mapping=json_struct)
    #customer = json_str
    #customer['id'] = customer_id
    return write_redis, 201

In [17]:
get()

[{b'customer_id': b'1234', b'name': b'full_name24', 'id': b'customer:1234'},
 {b'customer_id': b'123', b'name': b'full_name22', 'id': b'customer:123'}]

In [14]:
json_struct = {'customer_id':"1234", "name": "full_name24"}
post(json_struct)

{'customer_id': '1234', 'name': 'full_name24'}
key=customer:1234


(0, 201)

In [88]:
def get_img():
    key_mask = "raw_image:*"
    image_list = []
    for key in redis_client.scan_iter(key_mask):
        image_id = key 
        print(image_id)
        image = redis_client.get(key)
        image_list.append(image)
        #print(image)
    return image_list

def post_img(img_id, raw_image_bytes):
    
    key = "raw_image:" + img_id
    print(f"key={key}")
    write_redis = redis_client.set(key, raw_image_bytes)
    return write_redis, 201

In [89]:
img_list = get_img()
print(type(img_list))
print(len(img_list))
print(len(img_list[0]))
print(type(img_list[0]))

b'raw_image:12345'
<class 'list'>
1
227791
<class 'bytes'>


In [90]:
import mxnet as mx
fname = mx.test_utils.download(
    "https://github.com/dmlc/web-data/blob/master/mxnet/doc/tutorials/python/predict_image/cat.jpg?raw=true",
    "cat.jpg",
)

with open(fname, "rb") as f:
    payload = f.read()
    post_img("12345", payload)



key=raw_image:12345


In [67]:
with open(fname, "rb") as f:
    payload = f.read()
    write_redis = redis_client.set('other_raw_im_:1', payload)
payload_got_back = redis_client.get('other_raw_im:1')
print(type(payload_got_back))
print(len(payload_got_back))

<class 'bytes'>
227791


In [91]:
img_list = get_img()

b'raw_image:12345'


In [92]:
len(img_list)

1

In [63]:
payload # -- b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\

b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xe1\x00`Exif\x00\x00II*\x00\x08\x00\x00\x00\x02\x001\x01\x02\x00\x07\x00\x00\x00&\x00\x00\x00i\x87\x04\x00\x01\x00\x00\x00.\x00\x00\x00\x00\x00\x00\x00Google\x00\x00\x03\x00\x00\x90\x07\x00\x04\x00\x00\x000220\x02\xa0\x04\x00\x01\x00\x00\x00@\x06\x00\x00\x03\xa0\x04\x00\x01\x00\x00\x00\xb0\x04\x00\x00\x00\x00\x00\x00\xff\xdb\x00\x84\x00\x03\x02\x02\x08\x08\n\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x07\x07\x07\x08\x08\x08\x07\x07\x07\x07\x07\x07\x07\x07\x07\x07\x07\x07\x07\x07\x07\x07\x07\x07\x07\n\x07\x07\x07\x08\t\t\t\x07\x07\x0c\x0c\n\x08\x0c\x07\x08\t\x08\x01\x03\x04\x04\x06\x05\x06\n\x06\x06\n\x0c\x0c\x0c\x0c\r\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\xff\xc0\x00\x11\x08\x04\xb0\x06@\x03\x01"\x00\x02\x11\x01\x03\x11\x01\xff\xc4\x00\x1d\x00\

In [93]:
payload_got_back = img_list[0]
len(payload_got_back)

227791

In [94]:
# Input image is in bytearray, convert it to MXNet NDArray
img = mx.img.imdecode(payload_got_back)


# convert into format (batch, RGB, width, height)
img = mx.image.imresize(img, 224, 224)  # resize
img = img.transpose((2, 0, 1))  # Channel first
img = img.expand_dims(axis=0)  # batchify
img


[[[[ 24  25  25 ...  44  42  47]
   [ 26  24  24 ...  48  45  51]
   [ 25  26  25 ...  55  54  54]
   ...
   [ 36  34  35 ... 205 153 199]
   [ 33  36  40 ... 212 195 192]
   [ 43  47  49 ... 215 204 178]]

  [[ 57  56  58 ...  76  77  90]
   [ 59  55  57 ...  83  84  96]
   [ 57  57  58 ...  91  93 101]
   ...
   [ 47  45  46 ... 198 146 192]
   [ 48  51  52 ... 203 188 185]
   [ 61  66  62 ... 206 197 171]]

  [[ 50  50  51 ...  13  19  34]
   [ 52  49  50 ...  19  20  33]
   [ 50  51  51 ...  27  26  29]
   ...
   [ 17  11  14 ... 180 128 173]
   [ 12  14  13 ... 186 169 166]
   [ 21  21  17 ... 189 179 152]]]]
<NDArray 1x3x224x224 @cpu(0)>